### Vamos a probar la nueva data en español

In [2]:
import pandas as pd
import sys
import os
import re
sys.path.append(os.path.abspath('../scripts'))  #esto lo colocamos para que el python nos lea la ruta bien
from utils import process_null_values, tokenice_and_lemati # Importa la funcinn desde utils


In [2]:
MultiLeng = pd.read_csv('../data/spanish/train.csv')
MultiLeng.describe()

,Unnamed: 0,stars
count,1.200000e+06,1.200000e+06
mean,5.999995e+05,3.000000e+00
std,3.464103e+05,1.414214e+00
min,0.000000e+00,1.000000e+00
25%,2.999998e+05,2.000000e+00
50%,5.999995e+05,3.000000e+00
75%,8.999992e+05,4.000000e+00
max,1.199999e+06,5.000000e+00


In [3]:
DataS = pd.read_csv('../data_process/DataSp1r.csv')

In [3]:
MultiLeng.head()

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,0,de_0203609,product_de_0865382,reviewer_de_0267719,1,Armband ist leider nach 1 Jahr kaputt gegangen,Leider nach 1 Jahr kaputt,de,sports
1,1,de_0559494,product_de_0678997,reviewer_de_0783625,1,In der Lieferung war nur Ein Akku!,EINS statt ZWEI Akkus!!!,de,home_improvement
2,2,de_0238777,product_de_0372235,reviewer_de_0911426,1,"Ein Stern, weil gar keine geht nicht. Es hande...",Achtung Abzocke,de,drugstore
3,3,de_0477884,product_de_0719501,reviewer_de_0836478,1,"Dachte, das wären einfach etwas festere Binden...",Zu viel des Guten,de,drugstore
4,4,de_0270868,product_de_0022613,reviewer_de_0736276,1,Meine Kinder haben kaum damit gespielt und nac...,Qualität sehr schlecht,de,toy


### Eliminando fiilas innecesarias

In [4]:
MultiLeng = MultiLeng.drop(MultiLeng[(MultiLeng['language'] != 'en') & (MultiLeng['language'] != 'es')].index)

In [5]:
conteo_reseñas = MultiLeng.groupby('language').size()
conteo_reseñas

language
en    200000
es    200000
dtype: int64

## Limpiando datos

In [4]:
process_null_values(DataS)

Total de registros:
la columna processed_text tiene un Total de: 54 valores vacios
--------------------------------------------------------------------------------
Porcentaje de registros:
la columna processed_text tiene un Porcentaje de: 0.03461538461538461 valores vacios
--------------------------------------------------------------------------------
Si las columnas son irrelevantes, podemos hacer algo con sus datos, que te gustaria eliminarlos o los conservamos?


todas las filas con la columna processed_text han sido eliminadas
data limpiada maestro, que tengas un buen dia :)


## Eliminando columnas innecesarias

In [7]:
MultiLeng.drop(['review_id'], axis=1, inplace=True)
MultiLeng.drop(['product_id'], axis=1, inplace=True)
MultiLeng.drop(['reviewer_id'], axis=1, inplace=True)
MultiLeng.drop(['product_category'], axis=1, inplace=True)
MultiLeng.drop(['Unnamed: 0'], axis=1, inplace=True)


In [8]:
MultiLeng.head()

,stars,review_body,language
200000,1,Arrived broken. Manufacturer defect. Two of th...,en
200001,1,the cabinet dot were all detached from backing...,en
200002,1,I received my first order of this product and ...,en
200003,1,This product is a piece of shit. Do not buy. D...,en
200004,1,went through 3 in one day doesn't fit correct ...,en


### divideremos la data en espaniol e ingles, por si acaso la de ingles no es de ayuda

In [9]:
DataSpanish = MultiLeng[(MultiLeng['language'] == 'es')]
DataEnglish = MultiLeng[(MultiLeng['language'] == 'en')]


In [ ]:
DataSpanish.head()

,stars,review_body,language
400000,1,Nada bueno se me fue ka pantalla en menos de 8...,es
400001,1,"Horrible, nos tuvimos que comprar otro porque ...",es
400002,1,Te obligan a comprar dos unidades y te llega s...,es
400003,1,"No entro en descalificar al vendedor, solo pue...",es
400004,1,Llega tarde y co la talla equivocada,es


## VISUALIZAR DANTIDAD DE DE LOS PUNTAJES

In [21]:
#antes de pasar a hacer la lematizacion hare una eliminacion de muchos datos positivos que hay(no hay de otra)
#eliminar muchos registros para balancear la data
condition = DataS['stars'] == 1
num_to_remove = 1000
rows_to_remove = DataS[condition].head(num_to_remove)

# Eliminar esas filas del DataFrame original
DataS = DataS.drop(rows_to_remove.index)

DataS.describe()


,stars
count,115946.000000
mean,2.982535
std,1.420255
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,5.000000


In [22]:
conteo_reseñas = DataS.groupby('stars').size()
conteo_reseñas

stars
1    23995
2    22995
3    22984
4    22984
5    22988
dtype: int64

In [17]:
DataSpanish['processed_text'] = DataSpanish['review_body'].apply(lambda x: re.sub(r'[^\w\sáéíóúñÁÉÍÓÚÑ]', '', x))

In [23]:
DataS.head()

,stars,review_body,language,processed_text
8002,1,"No ha durado ni 3 semanas, sale agua por el la...",es,no durado semanas sale agua lateral desmonta p...
8003,1,Tardo una eternidad en llegar y no vale para m...,es,tardo eternidad llegar no vale dremel original...
8004,1,"Aunque el producto es el mismo, llego sin emba...",es,producto llego embalaje reído bolsa plástico n...
8005,1,Es un cajón de contrachapado con cuatro patas....,es,cajón contrachapado patas queda cojo personalm...
8006,1,Hace más de una semana q intento pedir la fact...,es,semana q intento pedir factura pedido enviado ...


In [3]:
# Procesar la columna 'review_body' en lote
DataS['processed_text'] = tokenice_and_lemati(DataS['review_body'], process=2, language='es')

In [10]:
DataS.head()

,stars,review_body,language,processed_text
0,1,No lee nada. Ni usb ni mp3 ni nada. Un engaño ...,es,no lee usb engaño tods regla no dejar qur vend...
1,1,En mi opinión es mejor el manual q venden en m...,es,opinión mejor manual q venden mercadona
2,1,"Era para regalar a mi hermano, dice que muy du...",es,regalar hermano muy duras difícil moldear toca...
3,1,Me costó un poko instalar no lo puedo utilizar,es,costó poko instalar no utilizar
4,1,"Muy mala tela. Se queda toda la pelusa pegada,...",es,muy mala tela queda pelusa pegada lava suelta ...


In [6]:
DataS.to_csv('../data_process/DataSp1r.csv', index=False)

In [32]:
from transformers import pipeline

# Cargar el modelo
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

def analizar_sentimiento(textos, batch_size=32):
    resultados = []
    
    # Procesar los textos en lotes
    for i in range(0, len(textos), batch_size):
        batch = textos[i:i + batch_size]  # Extraer lote
        predicciones = classifier(batch)  # Analizar el lote
        
        for resultado in predicciones:
            score = resultado["score"]
            label = resultado["label"]

            # Clasificar el sentimiento de forma similar a VADER
            if "5" in label:  
                resultados.append("contento")  # Muy positivo
            elif "4" in label:
                resultados.append("positivo")  # Positivo
            elif "1" in label:
                resultados.append("enojado")  # Muy negativo
            elif "2" in label:
                resultados.append("insatisfecho")  # Negativo
            elif "3" in label:
                resultados.append("neutral")  # Neutral
            else:
                resultados.append("neutral")  # Caso por defecto

    return resultados

Device set to use cpu


In [33]:
from transformers import pipeline

# Cargar el modelo de BETO optimizado
sentiment_analyzer = pipeline(
    "text-classification", 
    model="nlptown/bert-base-multilingual-uncased-sentiment",  
    device=-1  # CPU (-1) o GPU (0 si tienes CUDA)
)

def analizar_sentimiento_beto(textos):
    resultados = []
    
    # Procesar textos en lote para mayor velocidad
    predicciones = sentiment_analyzer(textos, truncation=True)
    
    for pred in predicciones:
        label = pred["label"]
        
        # Mapear etiquetas a categorías personalizadas
        if "5" in label:
            resultados.append("contento")  # Muy positivo
        elif "4" in label:
            resultados.append("positivo")  # Positivo
        elif "1" in label:
            resultados.append("enojado")  # Muy negativo
        elif "2" in label:
            resultados.append("insatisfecho")  # Negativo fuerte
        elif "3" in label:
            resultados.append("neutral")  # Neutral
        else:
            resultados.append("neutral")  # Caso por defecto
    
    return resultados


Device set to use cpu


In [54]:
import time
from transformers import pipeline

# Cargar modelo de Hugging Face
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-multilingual-cased", device=-1)

def analizar_sentimiento_distilbert(textos, batch_size=32):
    """
    Analiza el sentimiento de una lista de textos en español usando DistilBERT.

    :param textos: Lista de textos a analizar.
    :param batch_size: Tamaño del lote para mejorar rendimiento.
    :return: Lista con las categorías de sentimiento asignadas.
    """
    resultados = []
    inicio_tiempo = time.time()  # Inicia el cronómetro

    # Procesar en lotes
    for i in range(0, len(textos), batch_size):
        batch = textos[i:i + batch_size]
        predicciones = sentiment_analyzer(batch)

        for resultado in predicciones:
            label = resultado["label"]
            score = resultado["score"]

            # Asignar categorías de sentimiento (ajustadas manualmente)
            if "POSITIVE" in label and score >= 0.9:
                resultados.append("contento")  # Muy positivo
            elif "POSITIVE" in label:
                resultados.append("positivo")  # Positivo
            elif "NEGATIVE" in label and score >= 0.9:
                resultados.append("enojado")  # Muy negativo
            elif "NEGATIVE" in label:
                resultados.append("insatisfecho")  # Negativo
            else:
                resultados.append("neutral")  # Neutral

    fin_tiempo = time.time()  # Detiene el cronómetro
    print(f"Tiempo total de procesamiento: {fin_tiempo - inicio_tiempo:.2f} segundos")
    return resultados


In [34]:
data_muestra = DataS.head(1000)

In [35]:
textos = data_muestra["review_body"].tolist()

In [36]:
sentimientos = analizar_sentimiento_beto(textos)

In [37]:
data_muestra["sentiment"] = sentimientos

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14796\3657751988.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_muestra["sentiment"] = sentimientos


In [38]:
data_muestra.head()

,stars,review_body,language,processed_text,sentiment
8002,1,"No ha durado ni 3 semanas, sale agua por el la...",es,no durado semanas sale agua lateral desmonta p...,enojado
8003,1,Tardo una eternidad en llegar y no vale para m...,es,tardo eternidad llegar no vale dremel original...,enojado
8004,1,"Aunque el producto es el mismo, llego sin emba...",es,producto llego embalaje reído bolsa plástico n...,insatisfecho
8005,1,Es un cajón de contrachapado con cuatro patas....,es,cajón contrachapado patas queda cojo personalm...,insatisfecho
8006,1,Hace más de una semana q intento pedir la fact...,es,semana q intento pedir factura pedido enviado ...,enojado


In [44]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

data_muestra['sentiment'] = DataS['review_body'].apply(
    lambda x: "POS" if analyzer.polarity_scores(x)['compound'] > 0 else "NEG"
) 

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5412\2410624008.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_muestra['sentiment'] = DataS['review_body'].apply(


In [59]:
data_muestra.head(70)

,stars,review_body,language,processed_text,sentiment
6002,1,"NO ME HA LLEGADO ESTOS PENDIENTES, QUIERO PONE...",es,no llegado pendientes quiero reclamacion pendi...,neutral
6003,1,Me deja la espalda destrozada. No la puedo usa...,es,deja espalda destrozada no minutos seguidos de...,neutral
6004,1,Despues de un mes de la devolucion aun no se n...,es,mes devolucion no reloj dinero,neutral
6005,1,"No se ajusta bien, es más pequeño que el móvil...",es,no ajusta pequeño móvil compré no pega no repe...,neutral
6006,1,NO FUNCIONA CON PRODUCTOS APPLE ES UNA ESTAFA,es,no funciona productos apple estafa,neutral
...,...,...,...,...,...
6067,1,Lo peor que he visto...,es,peor visto,neutral
6068,1,Los compre con la ilusión de que iba a encontr...,es,compre ilusión iba encontrar utilizar no rompi...,neutral
6069,1,No lo recomiendo.. Experiencia nada satisfacto...,es,no recomiendo experiencia satisfactoria supone...,neutral
6070,1,No es un buen producto.,es,no producto,neutral


In [53]:
data_muestra.to_csv('../data_process/Muestra.csv', index=False)


In [ ]:
DataS['sentiment'] = DataS['review_body'].apply(analizar_pysentimiento)


KeyboardInterrupt: 